# Notebook pour la classification des affiches de cinéma

En classifiant les images, ce Notebook permet de séparer les différents visuels des affiches de cinéma: dessins, photographies ou hybride. Au préalable 3'000 images ont été labellisées selon les trois catégories. Ensuit, différents réseaus de neurones existant et entrainé sont repris pour notre tâche. Les dernières couches de sorties sont enlevées et remplacée par une sortie vide qui permettra la prédiction de nos données.

**Ce Notebook sert d'essaie aux différents modèles mis à jour dans l'état de l'art, pour le colloque Humanistica (Genève, 2022)**

*Plan du Notebook*

> Importer les données

> Charger et appliquer le modèle

> Visualiser les métriques

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
from PIL import ImageFile

%pip install tensorflow_addons
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import Precision, Recall
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.models import Model

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
plt.rcParams.update(new_rc_params)

## Importer les données

Cette étape demande d'avoir, au préalable, un dossier avec les données d'entrainement et de validation chacun séparé entre les trois catégories.

In [ ]:
# Chemins des données d'entraînement et de validation
train_data_dir = ""
validation_data_dir = ""
test_data_dir = ""
# les chemins sont bons?
print(os.path.isdir(train_data_dir), os.path.isdir(test_data_dir), os.path.isdir(validation_data_dir))

### Chargement, transformation et normalisaiton 
 Les arguments définissent une série de transformations aléatoires qui seront appliquées aux images générées pendant l'entraînement.

In [ ]:
#chargement des données

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2, 
    zoom_range=0.2, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True, 
    rotation_range=20,
    fill_mode="nearest")

val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_generator.classes),
                                        y = train_generator.classes                                                    
                                    )
class_weights = dict(zip(np.unique(train_generator.classes), class_weights))
class_weights

## Chargement du modèle

D'abord les modèles sont importés.  L'entrainement des couches est conservé. Seule la dernière couche est supprimée. Elle est remplacé par une couche vide qui permet de prédire trois catégories. 

Les modèles suivants ont été expérimentés:
- VGG16
- ResNet50
- DenseNet169
- InceptionV3
- MobileNetV2

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet169

In [ ]:
# Chargement du modèle pré-entraîné
base_model = MobileNetV2(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3))
#base_model.summary()

### Modification de la fin du modèle pré-entrainé

In [ ]:
# Ajout d'une couche d'agrégation pour réduire les dimensions des données
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
# Ajout d'une couche dense de sortie pour classer les images en trois catégories
predictions = Dense(3, activation='softmax')(x)

In [ ]:
# Congélation des couches du modèle pré-entraîné pour ne pas les entraîner
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Génération du modèle en utilisant les couches précédentes
model = Model(inputs=base_model.input, outputs=predictions)
#model.summary()

### Compilation et entraînement du modèle (+sauvegarde)

Le modèle est ensuite compilé, avec les métriques nécessaires. Finalement il est entrainé puis sauvegardé

In [ ]:
# Compilation du modèle en incluant différentes métriques
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=3)])

In [ ]:
# Entraînement du modèle
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    class_weight=class_weights
)

In [ ]:
# la date du jour

from datetime import date
 
today = date.today()
today = str(today).replace("-", "")
today2 = today[-2:] + today[4:-2] + today[2:-4]
print(today2)

In [ ]:
# Sauvegarde du modèle entraîné
nom_du_model = base_model.name+ "_"+ today2
model.save('/models/model_' +nom_du_model+'.h5')

## Enregistrement des performances du modèle

- "Accuracy"
- Perte ("Loss")
- Précision ("Precision")
- Rappel ("Recall")
- F1-score

Pour comprendre, sommairement, les choix de l'algorithme, les mauvaises prédicitions sont visualisées à l'aide d'une matrice de confusion, puis directement les images.

In [ ]:
# évaluer le modèle

results = model.evaluate(test_generator, steps=len(test_generator))
print("test loss, test acc:", results)

In [ ]:
# enregistrer

# Création du dictionnaire
data = {
    'model': nom_du_model,
    'test_loss': results[0],
    'test_accuracy': results[1],
    'test_precision': results[2],
    'test_recall': results[3],
    'test_f1_score': results[4],
}

# Conversion du dictionnaire en DataFrame
df = pd.DataFrame(data)

# Enregistrement du DataFrame dans un fichier CSV
df.to_csv(f'test_mesures_{nom_du_model}.csv')


### Prédire les images et visualiser les mauvaises prédictions

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 1. Après avoir entraîné votre modèle, effectuez les prédictions sur l'ensemble de validation

predicted_labels = model.predict(validation_generator)
predicted_labels = np.argmax(predicted_labels, axis=1)

# 2. Collectez les vraies étiquettes pour l'ensemble de validation

true_labels = validation_generator.classes

# 3. Créez la matrice de confusion

confusion_mat = confusion_matrix(true_labels, predicted_labels)

# 4. Visualisez la matrice de confusion avec un heatmap

class_names = list(validation_generator.class_indices.keys())

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Réelle')
plt.title('Matrice de Confusion (DenseNet169)')
plt.savefig(f'', dpi=200)
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Prédire les classes pour les images de validation
Y_pred = model.predict_generator(validation_generator, len(validation_generator))

# Convertir les prédictions en étiquettes de classe
y_pred = np.argmax(Y_pred, axis=1)

# Obtenir les noms des classes
class_names = list(train_generator.class_indices.keys())

# Obtenir les étiquettes de classe réelles
y_true = validation_generator.classes

# Afficher le rapport de classification
print(classification_report(y_true, y_pred, target_names=class_names))


In [1]:
# Obtenir les noms des fichiers d'images
filenames = validation_generator.filenames

# Identifier les images mal classées
misclassified_indices = np.where(y_pred != y_true)[0]

# Visualiser les images mal classées
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))

for i, index in enumerate(misclassified_indices[:4]):
    row = i // 2
    col = i % 2
    img = plt.imread(validation_data_dir + '/' + filenames[index])
    axs[row][col].imshow(img)
    axs[row][col].set_title(f"Vraie classe : {class_names[y_true[index]]}\nClasse prédite : {class_names[y_pred[index]]}")
    axs[row][col].axis('off')

plt.tight_layout()
plt.savefig(f"misclassified_{nom_du_model}.png", dpi=200)
plt.show()


NameError: name 'validation_generator' is not defined